<h3>Simple KNN</h3>

In [78]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
from sklearn.metrics import accuracy_score

In [79]:
data = load_iris()
X = data.data
y = data.target

print("the number of data points here is ",X.shape)
print("*"*20)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 20)
print("the shape of train data is ",X_train.shape)
print("the shape of test data is ",X_test.shape)

the number of data points here is  (150, 4)
********************
the shape of train data is  (130, 4)
the shape of test data is  (20, 4)


In [80]:
type(X_train)

numpy.ndarray

In [81]:
def sort_tuple(tup): #https://www.geeksforgeeks.org/python-program-to-sort-a-list-of-tuples-by-second-item/
    # convert the list of tuples to a numpy array with data type (object, int)
    arr = np.array(tup, dtype=[('col1', object), ('col2', int)])
    # get the indices that would sort the array based on the second column
    indices = np.argsort(arr['col1'])
    # use the resulting indices to sort the array
    sorted_arr = arr[indices]
    # convert the sorted numpy array back to a list of tuples
    sorted_tup = [(row['col1'], row['col2']) for row in sorted_arr]
    return sorted_tup

def get_final_outcome(outcome,max_value):
    for k,v in outcome.items():
        if v == max_value:
            return k
        


def knn_scratch(X_data,k = 2):

    y_pred_target = []
    for x_i in X_data:
        dist_y_label = []
        loop_cnt = 0
        for x_j in X_train:
#             if not np.array_equal(x_i,x_j) :
                dist_value = 0
                for d in range(x_i.shape[0]):
                    dist_value += ((x_i[d] - x_j[d]) * (x_i[d] - x_j[d]))
                dist_y_label.append((math.sqrt(dist_value),y_train[loop_cnt]))
                loop_cnt += 1
        sorted_outcome = sort_tuple(dist_y_label)
        final_outcome = sorted_outcome[0:k]

        outputs = []
        for m in final_outcome:
            outputs.append(m[1])
        outputs = dict(Counter(outputs))

        max_value = max(outputs.values())
        y_pred = get_final_outcome(outputs,max_value)
        y_pred_target.append(y_pred)
        
    return y_pred_target



In [96]:
pred_output = knn_scratch(X_train,k = 5)

In [97]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_train)
print(accuracy_score(y_train,y_pred))

0.9692307692307692


In [98]:
print("accuracy from knn implemented from scratch is ",accuracy_score(y_train,pred_output))
print("accuracy from knn implemted from sklearn is ",accuracy_score(y_train,y_pred))


accuracy from knn implemented from scratch is  0.9692307692307692
accuracy from knn implemted from sklearn is  0.9692307692307692


## Test data

In [99]:
pred_test = knn_scratch(X_test,k = 5)
y_pred = knn.predict(X_test)

print("Test data acc from scratch impl is ",accuracy_score(y_test,pred_test))
print("Test data acc from scratch impl is ",accuracy_score(y_test,y_pred))

Test data acc from scratch impl is  1.0
Test data acc from scratch impl is  1.0


<h3>Locality Sensitive hashing</h3>

In [82]:
from sklearn.preprocessing import StandardScaler

In [83]:
std = StandardScaler()
X_train_1 = std.fit_transform(X_train)

In [84]:
X_train_1 = np.round(X_train_1,4)

In [85]:
np.random.seed(10)
w_1 = np.random.normal(0,1,4)
w_2 = np.random.normal(0,1,4)
w_3 = np.random.normal(0,1,4)
print("the w1 vector is ",w_1)
print("the w2 vector is ",w_2)
w = [w_1,w_2]

the w1 vector is  [ 1.3315865   0.71527897 -1.54540029 -0.00838385]
the w2 vector is  [ 0.62133597 -0.72008556  0.26551159  0.10854853]


In [86]:
w

[array([ 1.3315865 ,  0.71527897, -1.54540029, -0.00838385]),
 array([ 0.62133597, -0.72008556,  0.26551159,  0.10854853])]

In [9]:
np.sign(np.dot(X_train[0],w[0]))

1.0

In [36]:
p = dict()
p[(1,2,3)]

1


In [27]:
t = (1,2,3)
t.update(4)
print(t)

AttributeError: 'tuple' object has no attribute 'update'

In [37]:
''.fromkeys({})

AttributeError: 'str' object has no attribute 'fromkeys'

In [87]:
slice_ind_dict = dict()
for x_i in X_train_1:
#     print("content of dict is ",slice_ind_dict)
    point_pos = []
    for plane in w:
        point_pos.append(np.sign(np.dot(x_i,plane)))
    point_pos = tuple(point_pos)
    if not point_pos in slice_ind_dict.keys():
        slice_ind_dict[point_pos] = []
        get_iterable = slice_ind_dict.get(point_pos)
        get_iterable.append(x_i)
    else:
        get_iterable = slice_ind_dict.get(point_pos)
        get_iterable.append(x_i)
        
    
print(slice_ind_dict.keys()) 
print(len(slice_ind_dict[(1.0, -1.0)]))
print(len(slice_ind_dict[(-1.0, 1.0)]))                                                                                                                                      
print(len(slice_ind_dict[(1.0, 1.0)]))
print(len(slice_ind_dict[(-1.0, -1.0)]))

dict_keys([(1.0, 1.0), (-1.0, 1.0), (-1.0, -1.0), (1.0, -1.0)])
40
61
23
6


In [103]:
y_pred_target = []
k = 3
for x_i in X_train_1:
    point_pos = []
    for plane in w:
        point_pos.append(np.sign(np.dot(x_i,plane)))
    point_pos = tuple(point_pos)
    get_all_neighbors = slice_ind_dict[point_pos]
    
    cos_dist_label = []
    for neighbor in get_all_neighbors:
        cos_sim = np.dot(x_i,neighbor)/((np.linalg.norm(x_i,2) * np.linalg.norm(neighbor,2)))
        cos_dist = 1-cos_sim
        y_i = data_df[(data_df["SL"] == neighbor[0]) & (data_df["SW"] == neighbor[1]) & (data_df["PL"] == neighbor[2]) & (data_df["PW"] == neighbor[3])]["label"].values[0] 
        print(y_i)
        cos_dist_label.append((cos_dist,y_i))
    sorted_outcome = sort_tuple(cos_dist_label)
    final_outcome = sorted_outcome[0:k]

    outputs = []
    for m in final_outcome:
        outputs.append(m[1])
    outputs = dict(Counter(outputs))

    max_value = max(outputs.values())
    y_pred = get_final_outcome(outputs,max_value)
    y_pred_target.append(y_pred)

1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
1
2
1
2
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
1
2
1
2
1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
1
2
1
2
1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
1
2
1
2
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
1
2
1
2
1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
1
2
1
2
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2


1
1
1
1
2
1
2
1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
1
2
1
2
1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
2
2
2
1
2
2
2
2
1
1
2
1
2
1
1
2
1
2
2
1
1
1
2
2
1
1
1
2
1
2
1
2
2
2
1
2
1
2
1
2
2
2
2
2
2
2
2
1
1
2
1
2
1
2
1
1
1
1
2
1
2
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
2
2
1
2
1
2
1
1
2
2
1
2
2
2
1
1
1
1
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [99]:
len(X_train_1)

130

In [101]:
len(y_pred_target)

130

In [104]:
accuracy_score(y_train,y_pred_target)

0.9230769230769231

In [89]:
data_df = pd.concat([pd.DataFrame(X_train_1,columns=["SL","SW","PL","PW"]),pd.DataFrame(y_train,columns=["label"])],axis = 1)

In [90]:
data_df.head()

,SL,SW,PL,PW,label
0,1.0628,-0.6343,0.5448,0.2233,1
1,-0.3519,-0.6343,0.6006,1.0008,2
2,0.9449,0.0567,0.9912,1.5191,2
3,1.1807,0.0567,0.8796,1.1304,2
4,1.4165,-0.1736,1.1586,1.1304,2


In [91]:
data_df.columns

Index(['SL', 'SW', 'PL', 'PW', 'label'], dtype='object')

In [66]:
data_df.loc[0,:]["SL"]

-0.4138

In [63]:
data_df[(data_df["SL"] == -0.41376443561368814) ]

,SL,SW,PL,PW,label
0,-0.413764,-1.766661,0.121558,0.129319,1
40,-0.413764,2.750577,-1.340600,-1.326790,0
115,-0.413764,-1.528911,0.009084,-0.135428,1
129,-0.413764,-1.291162,0.121558,0.129319,1


In [92]:
data_df["SL"] = np.round(data_df["SL"],4)
data_df["SW"] = np.round(data_df["SW"],4)
data_df["PL"] = np.round(data_df["PL"],4)
data_df["PW"] = np.round(data_df["PW"],4)

In [94]:
data_df["SL"]

0      1.0628
1     -0.3519
2      0.9449
3      1.1807
4      1.4165
        ...  
125    0.3555
126   -1.8844
127    0.9449
128    0.7091
129    0.4734
Name: SL, Length: 130, dtype: float64